In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import xlrd

# here you have to enter your actual API key from SimFin
api_key = "khJmsEQpgG1bK7Xl1vgXwyv9SCbrsBSP"

# list of tickers we want to get data for
tickers = ["AAPL",
            "NVDA",
            "WMT",
            "MSFT",
            "GOOG",
            "AMZN",
            "FB",
            "BRK.B",
            "JPM",
            "V",
            "JNJ",
            "XOM",
            "BAC",
            "PG",
            "T",
            "MA",
            "HD",
            "INTC",
            "DIS",
            "VZ",
            "UNH"]

# first: find SimFin IDs
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

# define time periods for financial statement data
statement_type = "cf"
time_periods = ["Q1", "Q2", "Q3", "Q4"]
year_start = 2015
year_end = 2019

# set up the XLSX writer
writer = pd.ExcelWriter("simfin_data.xlsx", engine='xlsxwriter')

data = {}
# get data for each ticker/id
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                # make time period identifier
                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0 and 'values' in statement_data:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]

        # fix the periods where no values were available
        len_target = len(d['Line Item'])
        if len_target > 0:
            for k, v in d.items():
                if len(v) != len_target:
                    d[k] = [None for _ in d['Line Item']]

    # convert to pandas dataframe
    df = pd.DataFrame(data=d)
    df["symbol"]=tickers[idx]
    # save in the XLSX file configured earlier
    df.to_excel(writer, sheet_name=tickers[idx])#,index=False)#get rid of the unnamed ON THE CONCAT

writer.save()
writer.close()

[111052, 172199, 239962, 59265, 18, 62747, 121021, None, 243510, 107326, 89661, 121214, 71941, 133209, 66790, 331628, 261673, 85652, 261356, 101219, 367714]


In [2]:
loc=("../project_22/simfin_data_cf.xlsx")
wb=xlrd.open_workbook(loc) #basically a pd.csv

In [3]:
df.to_csv(r'C:\Users\roel Cantu\Desktop\project_2.csv')
df.head(2)

,Line Item,Q1-2015,Q2-2015,Q3-2015,Q4-2015,Q1-2016,Q2-2016,Q3-2016,Q4-2016,Q1-2017,...,Q4-2017,Q1-2018,Q2-2018,Q3-2018,Q4-2018,Q1-2019,Q2-2019,Q3-2019,Q4-2019,symbol
0,Net Income/Starting Line,1413000000,1.585000e+09,1.618000e+09,1.252000e+09,1627000000,1.760000e+09,1.978000e+09,1.708000e+09,2191000000,...,3.721000e+09,2924000000,3.010000e+09,3.284000e+09,3.164000e+09,3557000000,None,None,None,UNH
1,Net Income,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,...,NaN,None,NaN,NaN,NaN,None,None,None,None,UNH


In [4]:
pd.concat(pd.read_excel("simfin_data_cf.xlsx",sheet_name=None), ignore_index=True, sort=True)
df.head(2)

,Line Item,Q1-2015,Q2-2015,Q3-2015,Q4-2015,Q1-2016,Q2-2016,Q3-2016,Q4-2016,Q1-2017,...,Q4-2017,Q1-2018,Q2-2018,Q3-2018,Q4-2018,Q1-2019,Q2-2019,Q3-2019,Q4-2019,symbol
0,Net Income/Starting Line,1413000000,1.585000e+09,1.618000e+09,1.252000e+09,1627000000,1.760000e+09,1.978000e+09,1.708000e+09,2191000000,...,3.721000e+09,2924000000,3.010000e+09,3.284000e+09,3.164000e+09,3557000000,None,None,None,UNH
1,Net Income,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,...,NaN,None,NaN,NaN,NaN,None,None,None,None,UNH


In [5]:
#transpose index
df.set_index("Line Item", inplace=True)

In [6]:
AAPL_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="AAPL")
AAPL_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AAPL_df.drop(["a"], axis=1, inplace=True)
AAPL_df=AAPL_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AAPL_df.set_index("Line Item",inplace=True)
AAPL_df.drop(columns= "symbol",inplace=True)
AAPL_df=AAPL_df.T
AAPL_df.head(2)

Line Item,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Stock-Based Compensation,Deferred Income Taxes,Change in Working Capital,(Increase) Decrease in Accounts Receivable,(Increase) Decrease in Inventories,Increase (Decrease) in Accounts Payable,Increase (Decrease) in Other,...,Dividends Paid,Cash From (Repayment of) Debt,"Cash From (Repayment of) Short Term Debt, net",Cash From (Repurchase of) Equity,Decrease in Capital Stock,Other Financing Activities,Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Net Cash Before FX,Net Changes in Cash
Q1-2015,1.802400e+10,2.575000e+09,3.085000e+09,888000000.0,2.197000e+09,1.003800e+10,7.510000e+08,-172000000.0,9.003000e+09,4.560000e+08,...,-2.801000e+09,1.076000e+09,-2.409000e+09,-4.950000e+09,-5.030000e+09,-248000000.0,-6.923000e+09,5.634000e+09,5.634000e+09,5.634000e+09
Q2-2015,1.356900e+10,2.479000e+09,6.090000e+08,927000000.0,-3.180000e+08,2.424000e+09,5.804000e+09,-113000000.0,-1.443100e+10,1.116400e+10,...,-2.743000e+09,7.748000e+09,-9.900000e+07,-6.741000e+09,-6.970000e+09,-3000000.0,-1.739000e+09,-4.989000e+09,-4.989000e+09,-4.989000e+09


In [ ]:
NVDA_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="NVDA")
NVDA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
NVDA_df.drop(["a"], axis=1, inplace=True)
NVDA_df=NVDA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
NVDA_df.set_index("Line Item",inplace=True)
NVDA_df.drop(columns= "symbol",inplace=True)
NVDA_df=NVDA_df.T
NVDA_df.head(2)

In [8]:
WMT_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="WMT")
WMT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
WMT_df.drop(["a"], axis=1, inplace=True)
WMT_df=WMT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
WMT_df.set_index("Line Item",inplace=True)
WMT_df.drop(columns= "symbol",inplace=True)
WMT_df=WMT_df.T
WMT_df.head(2)

Line Item,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Cash from Operating Activities,Change in Fixed Assets & Intangibles,Other Investing Activities,Cash from Investing Activities,Dividends Paid,Cash From (Repayment of) Debt,Cash From (Repurchase of) Equity,Other Financing Activities,Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Changes in Cash
Q1-2015,3.283000e+09,2.319000e+09,80000000.0,-1.236000e+09,4.446000e+09,-2.135000e+09,22000000.0,-2.113000e+09,-1.579000e+09,-1.613000e+09,-2.800000e+08,-2.230000e+08,-3.695000e+09,-1.362000e+09,-1.362000e+09,-14000000.0,-1.376000e+09
Q2-2015,3.635000e+09,2.339000e+09,56000000.0,-3.740000e+08,5.656000e+09,-2.622000e+09,-113000000.0,-2.489000e+09,-1.578000e+09,-1.230000e+09,-1.003000e+09,-1.268000e+09,-5.079000e+09,-1.912000e+09,-1.912000e+09,-96000000.0,-2.008000e+09


In [9]:
MSFT_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="MSFT")
MSFT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MSFT_df.drop(["a"], axis=1, inplace=True)
MSFT_df=MSFT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MSFT_df.set_index("Line Item",inplace=True)
MSFT_df.drop(columns= "symbol",inplace=True)
MSFT_df=MSFT_df.T
MSFT_df.head(2)

Line Item,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Stock-Based Compensation,Deferred Income Taxes,Other Non-Cash Adjustments,Change in Working Capital,(Increase) Decrease in Accounts Receivable,(Increase) Decrease in Inventories,Increase (Decrease) in Accounts Payable,...,"Cash From (Repayment of) Short Term Debt, net","Cash From (Repayment of) Long Term Debt, net",Repayments of Long Term Debt,Cash From Long Term Debt,Cash From (Repurchase of) Equity,Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Changes in Cash
Q1-2015,4.540000e+09,1.428000e+09,-1.014300e+10,144000000.0,301000000.0,-1.058800e+10,1.252900e+10,6.627000e+09,-4.830000e+08,-659000000.0,...,2.999000e+09,-1.500000e+09,-1.500000e+09,0.0,-2.672000e+09,-2.978000e+09,-2.361000e+09,-2.361000e+09,-6000000.0,-2.367000e+09
Q2-2015,5.863000e+09,1.521000e+09,-1.074900e+10,611000000.0,314000000.0,-1.167400e+10,7.705000e+09,-3.378000e+09,1.070000e+09,137000000.0,...,4.798000e+09,2.850000e+08,0.000000e+00,285000000.0,-2.024000e+09,5.340000e+08,1.580000e+08,1.580000e+08,-34000000.0,1.240000e+08


In [71]:
GOOG_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="GOOG")
GOOG_df.rename({"Unnamed: 0":"Line Item"}, axis="columns", inplace=True)
GOOG_df=GOOG_df.dropna(axis='columns', how="all").dropna(axis="rows", how="any")
GOOG_df.set_index("Line Item",inplace=True)
#GOOG_df = df.iloc[1:]
#GOOG_df.drop(columns= "symbol",inplace=True)
GOOG_df=GOOG_df.T
GOOG_df

Line Item,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash From Acquisitions & Divestitures,Other Investing Activities,Cash from Investing Activities,Cash From (Repayment of) Debt,Cash From (Repurchase of) Equity,Other Financing Activities,Cash from Financing Activities,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Changes in Cash
Q1 '15,3515,938,1590,679,6722,-2927,-3243,-64,-1070,-7304,-3,0,-493,-496,-1078,-293,-1371
Q2 '15,3931,1011,1206,948,7096,-2515,-3746,-78,1236,-5103,-3,0,-558,-561,1432,45,1477
Q3 '15,3979,1030,1274,-138,6145,-2373,-4139,-102,797,-5817,-3,0,-606,-609,-281,-104,-385
Q4 '15,4923,1153,2431,-1898,6609,-2100,-2661,8,-734,-5487,-14,-1780,-765,-2559,-1437,-82,-1519
Q1 '16,4207,1155,2468,-172,7658,-2428,-3668,-34,-115,-6245,-6,-2098,-807,-2911,-1498,60,-1438
Q2 '16,4877,1271,1713,1259,9120,-2123,-4812,-38,-200,-7173,-1042,-1595,-803,-3440,-1493,9,-1484
Q3 '16,5061,1377,1770,1637,9845,-2357,-8962,-252,-1474,-13045,-274,0,-815,-1089,-4289,68,-4221
Q4 '16,5333,1464,2040,576,9413,-3064,-1281,-662,305,-4702,-13,0,-879,-892,3819,-307,3512
Q1 '17,5426,1287,2914,-79,9548,-2467,-1111,-101,78,-3601,-18,-1127,221,-924,5023,191,5214
Q2 '17,3524,1424,2171,284,7403,-2818,-5021,-42,40,-7841,-38,-1618,-415,-2071,-2509,88,-2421


In [67]:
AMZN_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="AMZN")
AMZN_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AMZN_df.drop(["a"], axis=1, inplace=True)
AMZN_df=AMZN_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AMZN_df.set_index("Line Item",inplace=True)
AMZN_df.drop(columns= "symbol",inplace=True)
AMZN_df=AMZN_df.T
AMZN_df.head(2)

Line Item,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Stock-Based Compensation,Deferred Income Taxes,Other Non-Cash Adjustments,Change in Working Capital,(Increase) Decrease in Accounts Receivable,(Increase) Decrease in Inventories,Increase (Decrease) in Accounts Payable,...,Cash from Investing Activities,Cash From (Repayment of) Debt,"Cash From (Repayment of) Long Term Debt, net",Repayments of Long Term Debt,Cash From Long Term Debt,Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Changes in Cash
Q1-2015,-57000000.0,1.426000e+09,-644000000.0,407000000.0,-2000000.0,-1.049000e+09,-2.224000e+09,441000000.0,721000000.0,-4.249000e+09,...,-1.847000e+09,-674000000.0,-674000000.0,-857000000.0,183000000.0,-652000000.0,-3.998000e+09,-3.998000e+09,-322000000.0,-4.320000e+09
Q2-2015,92000000.0,1.504000e+09,-783000000.0,563000000.0,-43000000.0,-1.303000e+09,1.184000e+09,-430000000.0,-27000000.0,3.730000e+08,...,-1.376000e+09,-786000000.0,-786000000.0,-830000000.0,44000000.0,-691000000.0,-7.000000e+07,-7.000000e+07,102000000.0,3.200000e+07


In [ ]:
FB_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="FB")
FB_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
FB_df.drop(["a"], axis=1, inplace=True)
FB_df=FB_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
FB_df.set_index("Line Item",inplace=True)
FB_df.drop(columns= "symbol",inplace=True)
FB_df=FB_df.T
FB_df.head(2)

In [ ]:
JPM_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="JPM")
JPM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JPM_df.drop(["a"], axis=1, inplace=True)
JPM_df=JPM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JPM_df.set_index("Line Item",inplace=True)
JPM_df.drop(columns= "symbol",inplace=True)
JPM_df=JPM_df.T
JPM_df.head(2)

In [ ]:
V_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="V")
V_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
V_df.drop(["a"], axis=1, inplace=True)
V_df=V_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
V_df.set_index("Line Item",inplace=True)
V_df.drop(columns= "symbol",inplace=True)
V_df=V_df.T
V_df.head(2)

In [ ]:
JNJ_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="JNJ")
JNJ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JNJ_df.drop(["a"], axis=1, inplace=True)
JNJ_df=JNJ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JNJ_df.set_index("Line Item",inplace=True)
JNJ_df.drop(columns= "symbol",inplace=True)
JNJ_df=JNJ_df.T
JNJ_df.head(2)

In [ ]:
XOM_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="XOM")
XOM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
XOM_df.drop(["a"], axis=1, inplace=True)
XOM_df=XOM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
XOM_df.set_index("Line Item",inplace=True)
XOM_df.drop(columns= "symbol",inplace=True)
XOM_df=XOM_df.T
XOM_df.head(2)

In [ ]:
BAC_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="BAC")
BAC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
BAC_df.drop(["a"], axis=1, inplace=True)
BAC_df=BAC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
BAC_df.set_index("Line Item",inplace=True)
BAC_df.drop(columns= "symbol",inplace=True)
BAC_df=BAC_df.T
BAC_df.head(2)

In [ ]:
PG_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="PG")
PG_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
PG_df.drop(["a"], axis=1, inplace=True)
PG_df=PG_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
PG_df.set_index("Line Item",inplace=True)
PG_df.drop(columns= "symbol",inplace=True)
PG_df=PG_df.T
PG_df.head(2)

In [ ]:
T_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="T")
T_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
T_df.drop(["a"], axis=1, inplace=True)
T_df=T_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
T_df.set_index("Line Item",inplace=True)
T_df.drop(columns= "symbol",inplace=True)
T_df=T_df.T
T_df.head(2)

In [ ]:
HD_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="HD")
HD_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
HD_df.drop(["a"], axis=1, inplace=True)
HD_df=HD_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
HD_df.set_index("Line Item",inplace=True)
HD_df.drop(columns= "symbol",inplace=True)
HD_df=HD_df.T
HD_df.head(2)

In [ ]:
MA_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="MA")
MA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MA_df.drop(["a"], axis=1, inplace=True)
MA_df=MA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MA_df.set_index("Line Item",inplace=True)
MA_df.drop(columns= "symbol",inplace=True)
MA_df=MA_df.T
MA_df.head(2)

In [ ]:
INTC_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="INTC")
INTC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
INTC_df.drop(["a"], axis=1, inplace=True)
INTC_df=INTC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
INTC_df.set_index("Line Item",inplace=True)
INTC_df.drop(columns= "symbol",inplace=True)
INTC_df=INTC_df.T
INTC_df.head(2)

In [ ]:
DIS_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="DIS")
DIS_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
DIS_df.drop(["a"], axis=1, inplace=True)
DIS_df=DIS_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
DIS_df.set_index("Line Item",inplace=True)
DIS_df.drop(columns= "symbol",inplace=True)
DIS_df=DIS_df.T
DIS_df.head(2)

In [ ]:
VZ_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="VZ")
VZ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
VZ_df.drop(["a"], axis=1, inplace=True)
VZ_df=VZ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
VZ_df.set_index("Line Item",inplace=True)
VZ_df.drop(columns= "symbol",inplace=True)
VZ_df=VZ_df.T
VZ_df.head(2)

In [ ]:
UNH_df=pd.read_excel("simfin_data_cf.xlsx", sheet_name="UNH")
UNH_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
UNH_df.drop(["a"], axis=1, inplace=True)
UNH_df=UNH_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
UNH_df.set_index("Line Item",inplace=True)
UNH_df.drop(columns= "symbol",inplace=True)
UNH_df=UNH_df.T
UNH_df.head(2)